In [118]:
%%capture
%run 01_transaction.ipynb

In [119]:
from eth_account.messages import encode_defunct as encode_msg
import web3 as w3; w3 = w3.Account

Create a public key `pub_key`, private key `priv_key` pair. The public key is used as an address and the private key is used for signing txs.

In [120]:
def keys():
    acc = w3.create()
    return acc.privateKey.hex(), acc.address

priv_key,pub_key = keys()
ph(priv_key), ph(pub_key)

('💰 0x4a0fed97ddf022...516', '💎 0x28EC9Ff4bE9f8D...d69')

## Account

Holds a key pair and is used for signing txs.

In [129]:
class Account:
    def __init__(self): 
        self.priv, self.pub = keys()
        self.nonce = 0
        
    def sign(self, tx):
        self.nonce += 1
        m = encode_msg(bytes(tx))
        sig = w3.sign_message(m, self.priv)
        tx.sig = sig
        return tx
            
    def signed_tx(self, to, value, fee):
        tx = TX(self.pub, to.pub, value, fee, self.nonce)
        return self.sign(tx)
    
    def __str__(self): return ph(self.pub)

In [130]:
acc1 = Account(); print(acc1)
acc2 = Account(); print(acc2)

📋 0x6528D5f12FBF61...6C4
🔒 0xac8e135Df977d5...33F


### Transaction Signing
Signing a tx generates a signature that only the private key can produce. This can be later validated very efficiently.

In [131]:
tx        = TX(acc1.pub, acc2.pub, 12, 0.2, acc1.nonce)
tx_signed = acc1.sign(tx)

The signature contains the tx hash `message_hash` and its `signature`. We don't need `r`, `v` and `s` as they are specific to Ethereum. For more infos: https://medium.com/@angellopozo/ethereum-signing-and-validating-13a2d7cb0ee3

In [125]:
tx_signed.sig

SignedMessage(messageHash=HexBytes('0x333acac4bdb7fbf0ec79630073812e24b2513098b196d6d953777a51bc4183f0'), r=77147671813718251827399664742605040910979735664656298927729104149673286232610, s=77992857626443730793204208542630956044901492588271060359587194886504294054, v=27, signature=HexBytes('0xaa9008ec06ce2db5d979b21fef558931df476ba4966518e0ea47afd832d6ea22002c2473df46fcd1e17ba15d108367dda19671a9df23421a387f6b8a99bfa6a61b'))

Accounts can generate a signed tx easily with `signed_tx`.

In [133]:
tx = acc1.signed_tx(acc2, 7, 0.3); print(tx)

time:	Sat Mar 27 00:45:46 2021
from:	📋 0x6528D5f12FBF61...6C4
to:	🔒 0xac8e135Df977d5...33F
value:	7 ether
fee:	0.3 ether
nonce:	2
hash:	💁 1be38b283819df2e...610
signed:	true


### Transaction Validation

Validates if the tx was signed by the sender.

In [134]:
def val_tx(tx):
    if not hasattr(tx, 'sig'): return False
    m = encode_msg(bytes(tx))
    return w3.recover_message(m, signature=tx.sig.signature) == tx.fr

In [135]:
assert val_tx(tx_signed)

If anything in the tx is changed, like increasing the value to send, the signature should become invalid.

In [136]:
tx_signed.value = 30
assert not val_tx(tx_signed)